[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia-14/main?filepath=01%20Introduction%20to%20JuMP.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>

# Introduction to JuMP

JuMP.jl let's you solve mathematical optimization problems like contraint programming or schedule optimization.
Essentially it gives you a modeling language to interface numerous solvers.

In [ ]:
using JuMP, HiGHS
model = Model(HiGHS.Optimizer)

## The Basics

### Variables

In [ ]:
# named style
@variable(model, 0 <= v1 <= 6)  # default type is Float
@variable(model, v2, Bin)  # Int or Bin (Binary) are also valid

# anonymous style
v3 = @variable(model, integer=true, lower_bound=0)  # binary keyword

In [ ]:
# named variables are stored in the model
model[:v1]

In [ ]:
JuMP.object_dictionary(model)

### Constraints

Typically `<=` comparisons. Strict comparisons need to be modeled by an epsilon difference.

In [ ]:
# named constraint
@constraint(model, c1, 6v1 >= 100v2 + 3)

# anonymous constraint
c2 = @constraint(model, 7v3 >= 300(1-v2))

In [ ]:
# named constraints get also listed
JuMP.object_dictionary(model)

Different solvers can deal with different complexities. `HiGHS` can solve Mixed Integer Linear Programs, i.e. for example no quadratic constraints

### Objective

The goal, how the model shall be optimized

In [ ]:
@objective(model, Min, 12v1 + v2 + 20v3)

### Solve

In [ ]:
optimize!(model)

In [ ]:
# Always check whether the model was actually solved
is_solved_and_feasible(model) || error("Knock. Knock. I need your help.")

In [ ]:
# the value function can be used to extract solutions
(
    minimum = objective_value(model),
    variables = (value(v1), value(model[:v2]), value(v3)),
    constraints = (value(c1), value(c2)),
)

**👉 Your Challenge:** Change the constraints/objective such that `v1` becomes the largest value.

## N-Queens and indexed variables

Adapted from a tutorial by Matthew Helm and Mathieu Tanneau.

![N-Queens illustration](https://jump.dev/JuMP.jl/stable/assets/n_queens4.png)

In [ ]:
using LinearAlgebra
model_queens = Model(HiGHS.Optimizer)
N = 8

### Indexed Variables

This is actually super powerful - you can use arbitrary julia ranges or vectors for indices.
E.g. imagine indexing by date.

In [ ]:
@variable(model_queens, x[1:N, 1:N], Bin)

### Constraints

Similarly, constraints can be indexed. However here simple constraints are enough.

In [ ]:
# There must be exactly one queen in a given row/column
for i in 1:N
    # anonymous constraints get handy inside loops
    @constraint(model_queens, sum(x[i, :]) == 1)
    @constraint(model_queens, sum(x[:, i]) == 1)
end

In [ ]:
# There can only be one queen on any given diagonal
for i in -(N - 1):(N-1)
    # diag comes from LinearAlgebra
    @constraint(model_queens, sum(diag(x, i)) <= 1)
    @constraint(model_queens, sum(diag(reverse(x; dims = 1), i)) <= 1)
end

### Solve

This is how to combine JuMP model with data.

In [ ]:
fix(x[1, 2], 1)
fix(x[5, 3], 1)

now optimize given these given queen positions

In [ ]:
set_attribute(model_queens, "output_flag", false)  # hide HiGHS optimize! output 
optimize!(model_queens)
@assert is_solved_and_feasible(model_queens)
round.(Int, value.(x))

**👉 Your Challenge:** Change the initial fixed queens. What happens if you introduce an infeasible situation?

## Sudoku and constraint programming

Adapted from a tutorial by Iain Dunning.

![Sudoku illustration](https://jump.dev/JuMP.jl/stable/assets/partial_sudoku.png)

You can model Sudoku similar to N-Queens with Binary variables. Or you use contraint programming.


In [ ]:
model_sudoku = Model(HiGHS.Optimizer)

Instead of the binary variables, we directly define a 9x9 grid of integer values between 1 and 9:

In [ ]:
@variable(model_sudoku, 1 <= x[1:9, 1:9] <= 9, Int)

### Constraint Sets

JuMP has a couple of [special Sets](https://jump.dev/JuMP.jl/stable/tutorials/linear/constraint_programming/) which can be used for constraint programming.

In [ ]:
# values in each row/column must be all-different
# note, these are indexed constraints, a bit like a for loop
@constraint(model_sudoku, [i = 1:9], x[i, :] in MOI.AllDifferent(9))
@constraint(model_sudoku, [j = 1:9], x[:, j] in MOI.AllDifferent(9));

In [ ]:
# values in each 3x3 sub-grid must be all-different
for i in (0, 3, 6), j in (0, 3, 6)
    @constraint(model_sudoku, vec(x[i.+(1:3), j.+(1:3)]) in MOI.AllDifferent(9))
end

### Solve

In [ ]:
set_attribute(model_sudoku, "output_flag", false)
optimize!(model_sudoku)
@assert is_solved_and_feasible(model_sudoku)
round.(Int, value.(x))

### Fix initial values

**👉 Your Challenge:** Similarly to N-Queens, fix the given numbers for the Sudoku example.

In [ ]:
# Your Space
# ...

# Next: [02 Tips and Tricks and DisjunctiveProgramming](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia-14/main?filepath=02%20Tips%20and%20Tricks%20and%20DisjunctiveProgramming.ipynb)


For questions or suggestions please contact me at stephan.sahm@jolin.io

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>
